In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# データ読み込み

In [ ]:
import pandas as pd
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")

# データの確認
## 欠損値（各項目大体1.5%くらい）

In [ ]:
print("train:")
print(" total: {}".format(len(train)))
for col in train.columns:
    cnt = train[col].isnull().sum()
    if cnt:
        print(" {} {}".format(col, cnt))
        
print("test:")
print(" total: {}".format(len(test)))
for col in test.columns:
    cnt = test[col].isnull().sum()
    if cnt:
        print(" {} {}".format(col, cnt))

## 基礎統計量（この辺を見てスケーリングや欠損値補完をすると良さそう）

In [ ]:
train.describe()

## 正例と負例の割合（概ね均等）

In [ ]:
train["claim"].value_counts()

## 特徴量間の相関も見てみる(最大でも0.02とかなので気にしなくて良さそう)

In [ ]:
import seaborn as sns
cor = train.corr()
sns.heatmap(cor, cmap= sns.color_palette('coolwarm', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)

In [ ]:
max(list(cor.apply(lambda x: x.sort_values(key=lambda y:abs(y))[-2])))

# 前処理
## 欠損値を埋めてみる(とりあえず平均値で埋めた→スコアが悪化したのでやめ。良い方法を考える必要あり)

In [ ]:
# for col in test.columns:
#     test[col] = test[col].fillna(test[col].mean())
# for col in train.columns:
#     train[col] = train[col].fillna(train[col].mean())

## 外れ値の除去、スケーリングなど(ToDo)

# 特徴量作成
## featuretoolsを使ってみる

In [ ]:
# pip installなしでいけた。ありがたい
import featuretools as ft

In [ ]:
# featuretoolsに食わせる前にtest, trainをくっつける(両方に同じ処理をしたいため)
test['claim'] = np.nan
data = train.append(test, sort = True)

In [ ]:
# このコード作成時点でfeaturetools v1.0.0
# v1.0.0では　entity_from_dataframe　は廃止になった？
# 以下のソースコードに使い方がコメントアウトしてあるので、それに従う
# https://github.com/alteryx/featuretools/blob/main/featuretools/entityset/entityset.py

important_feature = ['f41', 'f71', 'f35', 'f36', 'f30', 'f48', 'f58', 'f78', 'f80', 'f22']

# dataframe全てを食わせるとメモリが溢れるので重要な特徴量10個に絞った
dataframes = {
    'data': (data.loc[:,important_feature+["id"]], "id")
}

# 特徴量を作成する前にentity setを定義する必要がある
# 各entityはdataframeをもとに作成できる（引数dataframesで指定）
# entityは複数持たせて、relationを貼ることができる。今回はとりあえず一つだけ指定
es = ft.EntitySet(
    'tabsep2021',
    dataframes
)

es

In [ ]:
# Deep Feature Synthesis(dfs)を実行、これをやることで特徴量を追加してくれる
# trans_primitivesで追加する特徴量の作り方（今回は差分と和）を指定
# entityが一つの場合、trans_primitivesを指定しないと追加特徴量は生成されない
feature_matrix, features_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="data",
    trans_primitives=['add_numeric', 'subtract_numeric']
)
feature_matrix.head(5)

In [ ]:
# 作成した特徴量を元のdataframeにマージ
data_edit = pd.merge(data, feature_matrix.drop(important_feature, axis=1), on='id')
data_edit.head(5)

# メモリ節約のため不要なオブジェクトを削除
del data
del feature_matrix

In [ ]:
# 特徴量の確認
list(data_edit.columns)

In [ ]:
# test, trainに分割
train = data_edit[data_edit['claim'].notnull()].reset_index()
test = data_edit[data_edit['claim'].isnull()].reset_index()

# モデル作成
## とりあえずLightGBMで

In [ ]:
train_X = train.drop(columns='claim')
train_Y = train['claim']

test_X = test.drop(columns='claim')

import lightgbm as lgb

# モデルの学習
model = lgb.LGBMClassifier() # モデルのインスタンスの作成
model.fit(train_X, train_Y) # モデルの学習

# テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率] を返す)
pred_Y = model.predict_proba(test_X)


In [ ]:
# submissionファイルの作成
# サンプルの目的変数の列を上書きすると楽
pred_1 = [pred_1 for pred_0, pred_1 in pred_Y]
submission = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")
submission['claim'] = pred_1
submission.to_csv("submission.csv",index=False)

## 各変数の重要度を見てみる

In [ ]:
import matplotlib.pyplot as plt

plt.bar(list(train_X.columns[:]), model.feature_importances_)

In [ ]:
imp_col = []
for imp, col in zip(model.feature_importances_, train_X.columns[1:]):
    imp_col.append((imp, col))
imp_col.sort(reverse=True, key=lambda x:x[0])

# 重要度の高い特徴量上位10件
important_feature = [col for imp, col in imp_col[:10]]
important_feature

In [ ]:
# 全部の特徴量の重要度
# 追加特徴量の重要度が低い、残念、、
imp_col

# モデル評価（ToDo　本当はクロスヴァリデーションして提出前に精度を確認した方が良い）